In [1]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np

Current working directory was changed to: C:\Users\viniciusbarros\Documents\GitHub\Previsor\App


In [2]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 29/12/22 - Horário: 11:38:16.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:06.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:10.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:20.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:42:20.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:35.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:48.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:44:27.  Portanto não foi baixado novamente.
vrapeelo

In [26]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)

In [4]:
monitoramento_usina = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[['IdeUsinaOutorga','NomUsina', 'SigTipoGeracao', 'CodCegFormatado']]

In [5]:
cols = ['IdeUsinaOutorga','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DthEnvio' : 'Data_Envio',
        'NomUsina' : 'usina_nome',
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)

In [46]:
cols = ['DthEnvio','IdeUsinaOutorga']
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols]


cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)


cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)

input_rapeel = pd.merge(mercado,ambiental_datas,on=['IdeUsinaOutorga','DthEnvio'],how="inner")
input_rapeel = pd.merge(input_rapeel,acesso_contratos,on=['IdeUsinaOutorga','DthEnvio'],how="inner")

In [45]:
input_rapee

,DthEnvio,IdeUsinaOutorga,DatValidadeLI,DatValidadeLP,DatValidadeLO
0,2019-11-04 13:28:22.543,27969,2018-07-28,NaT,NaT
1,2019-11-05 13:35:26.100,1997,NaT,2011-11-17,2024-01-22
2,2019-11-14 14:37:53.673,32923,2021-07-23,2016-07-17,2023-09-27
3,2019-11-05 17:34:59.677,30938,2019-10-16,2014-10-11,NaT
4,2019-11-05 17:17:39.770,324,2018-03-05,2002-12-11,NaT
...,...,...,...,...,...
60027,2022-12-14 17:41:40.507,51641,NaT,2023-03-31,NaT
60028,2022-12-14 17:44:27.333,51642,NaT,2023-03-31,NaT
60029,2022-12-14 17:45:57.813,51643,NaT,2023-03-31,NaT
60030,2022-12-14 17:47:06.843,51688,NaT,2023-04-05,NaT


In [11]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga","DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada"]
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols]

In [12]:
# vInputCronograma tbm implementado,DATASREAL (LABEL="Idempreendimento" AS Idempreendimento)
cols = ["DatRealizacaoIII","DatRealizacaoV","DatRealizacaoXIV","DatRealizacaoXII","IdeUsinaOutorga","DatRealizacaoIV","DatRealizacaoIX","DatRealizacaoXI","DatRealizacaoX","DthEnvio","DatRealizacaoVI","DatRealizacaoII","CodCeg","DatRealizacaoXIII","DatRealizacaoVII"]
rename_cols = {'DatRealizacaoIII': 'IO_real','DatRealizacaoXIV': 'Conc_Trans_REAL','DatRealizacaoXII': 'Comiss_Real','DatRealizacaoIX': 'DesvRio_real','DatRealizacaoXI': 'Ench_Real','DthEnvio': 'Data_Envio','DatRealizacaoII': 'canteiroReal','DatRealizacaoXIII': 'IOTrans_Real','DatRealizacaoVII': 'ME_Real_conc_eol'}
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates()[cols].rename(columns=rename_cols)
vcronograma["classe"] = vcronograma.CodCeg.str.slice(3,5)
vcronograma["CC_real"] = pd.to_datetime(pd.NA)
vcronograma.loc[vcronograma.classe == "PH","CC_real"] = vcronograma.DatRealizacaoIV
vcronograma.loc[vcronograma.classe == "CV","CC_real"] = vcronograma.DatRealizacaoV
vcronograma["ME_real"] = vcronograma.DatRealizacaoX
vcronograma.loc[vcronograma.classe == "CV","ME_real"] = vcronograma.DatRealizacaoVI

In [ ]:
cols = ["Data_Envio","Comiss_Real","canteiroReal","prev_IO","ME_Real_conc_eol","Conc_Trans_REAL","CC_real","DesvRio_real","Ench_Real","IdeUsinaOutorga","ME_real","IO_real","IOTrans_Real"]
realcronogramarapeel[cols].rename(columns=rename_cols)

In [19]:
#WORK.UG_Real n serve pra nd, inputug tbm n 
real_cronograma_rapeel = pd.merge(vcronograma,monitoramentoleilao,on=['IdeUsinaOutorga','Data_Envio'])

KeyError: 'Data_Envio'

In [21]:
monitoramentoleilao.monitoramentoleilao

AttributeError: 'DataFrame' object has no attribute 'monitoramentoleilao'

In [20]:
vcronograma.Data_Envio

0       2019-11-04 13:28:22.543
1       2019-11-05 13:35:26.100
2       2019-11-05 17:17:39.770
3       2019-11-05 17:34:59.677
4       2019-11-05 19:18:40.620
                  ...          
39266   2022-12-14 17:41:40.507
39267   2022-12-14 17:44:27.333
39268   2022-12-14 17:45:57.813
39269   2022-12-14 17:47:06.843
39270   2022-12-14 17:48:00.447
Name: Data_Envio, Length: 39271, dtype: datetime64[ns]